In [1]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn import tree

In [2]:
lending_data = pd.read_csv("LoanStats_2019Q1.csv")

FileNotFoundError: [Errno 2] File b'LoanStats_2019Q1.csv' does not exist: b'LoanStats_2019Q1.csv'

In [ ]:
lending_data.head()

In [ ]:
print(lending_data.keys())

In [ ]:
lending_data.info()

In [ ]:
lending_data.describe()

In [ ]:
lending_data.isnull().values.any()
lending_data.isnull().sum()

In [ ]:
print(lending_data.tail())

In [ ]:

lending_data = lending_data

# Inspect the shape
print(lending_data.shape)

In [ ]:
lending_data["int_rate"]=lending_data["int_rate"].str.replace(r"%","")
lending_data["revol_util"]=lending_data["revol_util"].str.replace(r"%","")
# lending_data["zip_code"]=lending_data["zip_code"].str.replace(r"xx","")
lending_data.head()

In [3]:
lending_df = lending_data[["loan_status", "loan_amnt", "int_rate",'grade', "annual_inc","dti", "delinq_2yrs", "inq_last_6mths", "open_acc", "pub_rec", "revol_bal", "revol_util", "total_acc"]]
status = lending_df.groupby("loan_status").count()
status

NameError: name 'lending_data' is not defined

In [4]:
status.describe()

NameError: name 'status' is not defined

In [13]:
status.isnull().values.any()
status.isnull().sum()

loan_amnt         0
int_rate          0
grade             0
annual_inc        0
dti               0
delinq_2yrs       0
inq_last_6mths    0
open_acc          0
pub_rec           0
revol_bal         0
revol_util        0
total_acc         0
dtype: int64

In [14]:
#Create the upsample and downsample
small_sample = lending_df[(lending_df["loan_status"] == "Charged Off") | (lending_df["loan_status"] == "Late (31-120 days)")]
large_sample = lending_df[(lending_df["loan_status"] == "Fully Paid")]

In [15]:
small_sample.head()
large_sample.head()

,loan_status,loan_amnt,int_rate,grade,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc
64,Fully Paid,25200.0,15.57,C,65000.0,15.18,0.0,0.0,4.0,0.0,17271.0,78.50,16.0
87,Fully Paid,17500.0,17.19,C,82000.0,15.70,1.0,0.0,12.0,0.0,15296.0,83.60,27.0
91,Fully Paid,10000.0,13.90,C,90000.0,7.92,0.0,1.0,5.0,0.0,9898.0,58.60,6.0
105,Fully Paid,13750.0,28.80,D,17808.0,30.12,0.0,2.0,6.0,0.0,16814.0,87.60,8.0
133,Fully Paid,10000.0,12.40,B,38000.0,38.66,0.0,0.0,7.0,0.0,8334.0,49,18.0


In [16]:
small_sample = small_sample.dropna()

In [17]:
large_sample = large_sample.dropna()

In [18]:
small_sample.isnull().values.any()
large_sample.isnull().sum()

loan_status       0
loan_amnt         0
int_rate          0
grade             0
annual_inc        0
dti               0
delinq_2yrs       0
inq_last_6mths    0
open_acc          0
pub_rec           0
revol_bal         0
revol_util        0
total_acc         0
dtype: int64

In [19]:
n_class0 = len(small_sample)
n_class1 = len(large_sample)

under = large_sample.sample(n_class0)
sample_df = pd.concat([under, small_sample], axis=0)

len(sample_df)

1930

In [27]:
X = sample_df[["loan_status", "loan_amnt", "int_rate",'grade', "annual_inc","dti", "delinq_2yrs", "inq_last_6mths", "open_acc", "pub_rec", "revol_bal", "revol_util", "total_acc"]]
y = sample_df["loan_status"].values.reshape(-1,1)
print(X.shape, y.shape)

(1930, 13) (1930, 1)


In [28]:
sample_df["loan_category"] = np.where(sample_df["loan_status"] == "Fully Paid", 1, 0)
sample_df.head()

,loan_status,loan_amnt,int_rate,grade,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,loan_category
101062,Fully Paid,3000.0,10.33,B,22000.0,4.80,0.0,1.0,5.0,0.0,3322.0,21.30,8.0,1
113809,Fully Paid,40000.0,10.33,B,102000.0,11.32,0.0,0.0,23.0,1.0,991.0,2.50,46.0,1
32145,Fully Paid,13500.0,8.81,A,49500.0,13.14,0.0,0.0,23.0,0.0,4566.0,14.80,34.0,1
76327,Fully Paid,22000.0,8.19,A,148000.0,16.95,0.0,0.0,7.0,0.0,14619.0,69.30,19.0,1
34016,Fully Paid,40000.0,15.57,C,118000.0,10.74,0.0,0.0,7.0,0.0,4423.0,19.80,19.0,1


In [29]:
data_binary_encoded = pd.get_dummies(sample_df, columns=["loan_status"])
data_binary_encoded.head()

,loan_amnt,int_rate,grade,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,loan_category,loan_status_Charged Off,loan_status_Fully Paid,loan_status_Late (31-120 days)
101062,3000.0,10.33,B,22000.0,4.80,0.0,1.0,5.0,0.0,3322.0,21.30,8.0,1,0,1,0
113809,40000.0,10.33,B,102000.0,11.32,0.0,0.0,23.0,1.0,991.0,2.50,46.0,1,0,1,0
32145,13500.0,8.81,A,49500.0,13.14,0.0,0.0,23.0,0.0,4566.0,14.80,34.0,1,0,1,0
76327,22000.0,8.19,A,148000.0,16.95,0.0,0.0,7.0,0.0,14619.0,69.30,19.0,1,0,1,0
34016,40000.0,15.57,C,118000.0,10.74,0.0,0.0,7.0,0.0,4423.0,19.80,19.0,1,0,1,0


In [30]:
# sample_df["loan_category"] = np.where(sample_df["loan_status"] == "Fully Paid", 1, 0)
# sample_df.head()

In [31]:
data = sample_df.drop("loan_status", axis=1)
feature_names = data.columns
data.head()

,loan_amnt,int_rate,grade,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,loan_category
101062,3000.0,10.33,B,22000.0,4.80,0.0,1.0,5.0,0.0,3322.0,21.30,8.0,1
113809,40000.0,10.33,B,102000.0,11.32,0.0,0.0,23.0,1.0,991.0,2.50,46.0,1
32145,13500.0,8.81,A,49500.0,13.14,0.0,0.0,23.0,0.0,4566.0,14.80,34.0,1
76327,22000.0,8.19,A,148000.0,16.95,0.0,0.0,7.0,0.0,14619.0,69.30,19.0,1
34016,40000.0,15.57,C,118000.0,10.74,0.0,0.0,7.0,0.0,4423.0,19.80,19.0,1


In [32]:
# import seaborn as sns
# sns.pairplot(data)

In [33]:
#2/3 YOUR ORIGINAL DATA SET AS THE TRAINING SET
#1/3 remains will compose the test set.
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)#EXPLIT THE DATA IN TO ???

In [34]:
clf = tree.DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)
clf.score(X_test, y_test)

ValueError: could not convert string to float: 'Late (31-120 days)'

In [ ]:
# Print the r2 score for the test data
# YOUR CODE HERE
print(f"Training Data Score: {clf.score(X_train, y_train)}")
print(f"Testing Data Score: {clf.score(X_test, y_test)}")

In [ ]:
clusters = clf.predict(X_test)
clusters

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200) #n_estimator is number of tree to be used in the forest
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test) 

In [ ]:
importances = rf.feature_importances_
importances

In [ ]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

In [ ]:
import seaborn as sns
sns.heatmap(data.corr(), annot=True)